In [306]:
# Combine all data from P1, P2 into final CCRI layer

In [307]:
import os
import pandas as pd
import geopandas as gpd
import glob
import numpy as np

In [308]:
# Query data dir (avoiding hard-coding paths when working between users)
#/content/drive/MyDrive/CCRI/ccri_repo/data
data_dir = '/content/drive/MyDrive/CCRI/ccri_repo/data'
print(data_dir)

/content/drive/MyDrive/CCRI/ccri_repo/data


In [309]:
# Check that data dir was input correctly
data_dir

'/content/drive/MyDrive/CCRI/ccri_repo/data'

In [310]:
# Read in all the data we need
# -- exposure files
p1_exposure_file = pd.read_csv('{}/CCRI_results_misc/Merged_Exposure_Data.csv'.format(data_dir))
p2_exposure_file = pd.read_csv('{}/CCRI_results_misc/P2_Merged_Normalized_avg.csv'.format(data_dir))
p1p2_scores = pd.read_csv('{}/CCRI_results_misc/p1_p2_avg_ccri.csv'.format(data_dir))

# -- attribute files
wb_income = pd.read_csv('{}/misc/WB_INCOME.csv'.format(data_dir))
unicef_ro = pd.read_csv('{}/misc/UNICEF_PROG_REG_GLOBAL.csv'.format(data_dir))

# -- population files
childpop = pd.read_csv('{}/CCRI_results_misc/child_pop_sum_adm0.csv'.format(data_dir))
#worldpop = pd.read_csv('{}/Misc/World_Population_ByAOI_adm0.csv'.format(data_dir))

# -- boundary file
adm0 = gpd.read_file('{}/misc/adm0_boundaries_simple.geojson'.format(data_dir))

# -- fragile codes
fragile = pd.read_csv('{}/misc/List of fragile context (2025).csv'.format(data_dir))

# -- component vals
p1_components = pd.read_csv('{}/CCRI_results_misc/p1_group_mean.csv'.format(data_dir))
p2_components = pd.read_csv('{}/CCRI_results_misc/p2_group_mean.csv'.format(data_dir))

In [311]:
# Renaming some columns
p1_exposure_file.columns = [col.replace('_absolute', '_abs_norm') if '_absolute' in col else col for col in p1_exposure_file.columns]
p1_exposure_file.columns = [col.replace('_relative', '_rel_norm') if '_relative' in col else col for col in p1_exposure_file.columns]

In [312]:
# Merge P exposures by ISO3
merged_P = (p1_exposure_file.merge(p2_exposure_file, on='iso3', how='left'))
all_P = (merged_P.merge(p1p2_scores, on='iso3', how='left'))

In [313]:
all_P = all_P.drop(columns=['P2_arithmetic_avg_y', 'rank_reverse_x'])
all_P = all_P.rename(columns={'P2_arithmetic_avg_x': 'P2_arithmetic_avg', 'rank_reverse_y': 'rank_reverse'})

In [314]:
# Add WB income
wb_income = wb_income[['Region_Code', 'ISO3Code']]
df = (all_P.merge(wb_income, left_on='iso3', right_on='ISO3Code', how='left').drop('ISO3Code', axis=1).rename(columns={'Region_Code': 'wb_income'}))

In [315]:
# Add Regional Office
unicef_ro = unicef_ro[['Region_Code','ISO3Code']]
df = (df.merge(unicef_ro, left_on='iso3', right_on='ISO3Code', how='left').drop('ISO3Code', axis=1).rename(columns={'Region_Code': 'unicef_ro'}))

In [316]:
# Add population data
# -- Take the childpop data from the geojson of p1p2 avg
gdf = gpd.read_file('{}/CCRI_results_misc/p1_p2_avg_ccri.geojson'.format(data_dir))
df_grouped = gdf[['ISO3', 'child_population_total', 'population_total']].groupby('ISO3', as_index=False).mean()
df_w_childpop = (df.merge(df_grouped, left_on=['iso3'], right_on=['ISO3'], how='left').rename(columns={'child_population_total': 'u18_pop'})).drop(columns=['ISO3'])

#df_w_childpop = (df.merge(childpop, left_on='iso3', right_on='ISO3', how='left').rename(columns={'child_population': 'u18_pop'})).drop(columns=['child_population_gpw'])
#df_w_childpop['u18_pop'] = df_w_childpop['u18_pop'].astype(int)

In [317]:
df_w_childpop = df_w_childpop.rename(columns={'iso3':'ISO3'})

In [318]:
'''
# Using World_Population_ByAOI_adm0.csv file -> removing for now to use the population data from the exposure files to keep consistent
df_w_allpop = (df_w_childpop.merge(worldpop, left_on='name', right_on='adm0_name', how='left').rename(columns={'sum': 'total_pop'})).drop(columns=['iso3','adm0_name'])

# Set population data as integer
df_w_allpop['u18_pop'] = df_w_allpop['u18_pop'].astype(int)
df_w_allpop['total_pop'] = df_w_allpop['total_pop'].astype(int)
'''

"\n# Using World_Population_ByAOI_adm0.csv file -> removing for now to use the population data from the exposure files to keep consistent\ndf_w_allpop = (df_w_childpop.merge(worldpop, left_on='name', right_on='adm0_name', how='left').rename(columns={'sum': 'total_pop'})).drop(columns=['iso3','adm0_name'])\n\n# Set population data as integer\ndf_w_allpop['u18_pop'] = df_w_allpop['u18_pop'].astype(int)\ndf_w_allpop['total_pop'] = df_w_allpop['total_pop'].astype(int)\n"

In [319]:
# Using simplified boundaries for geometry
adm0 = adm0[['ISO3', 'name', 'ucode','uuid','geometry','type']]
df_combined = (df_w_childpop.merge(adm0, left_on=['ISO3', 'adm0_name'], right_on=['ISO3', 'name'], how='left'))

In [320]:
# Grabbing actual exposure numbers
# Define file paths
exposure_path = "{}/p1_exposure".format(data_dir)

# Get all CSV files for exposure
exposure_files = glob.glob(os.path.join(exposure_path, "*.csv"))

# Initialize empty list for processed data
exposure_data_list = []

### **Process Each File in One Loop**
for file in exposure_files:
    df = pd.read_csv(file)  # Read full file to check available columns
    filename_only = os.path.basename(file)
    hazard_name = '_'.join(filename_only.split('_')[:2])  # Extract hazard name

    # Ensure required columns exist
    required_cols = {'iso3', 'adm0_name', 'child_population_exposed'}
    if not required_cols.issubset(df.columns):
        print(f"Skipping {file}: Missing columns {required_cols - set(df.columns)}")
        continue  # Skip if required columns are missing

    df.dropna(subset=['child_population_exposed'], inplace=True)

    # Compute relative exposure (%)
    df['{}_rel'.format(hazard_name)] = np.where(
        (df['child_population_total'] > 0) & (~df['child_population_total'].isna()),
        (df['child_population_exposed'] / df['child_population_total']) * 100,
        0
    )

    # Rename to hazard
    df = df.rename(columns={'child_population_exposed': '{}_abs'.format(hazard_name)})
    df = df.drop(columns=['child_population_total', 'population_total'])

    exposure_data_list.append(df)

In [321]:
merged_exposure_df = pd.concat(exposure_data_list, axis=1).drop_duplicates(subset=['iso3', 'adm0_name'])
# Ensure no duplicate columns before merging
merged_exposure_df = merged_exposure_df.loc[:, ~merged_exposure_df.columns.duplicated()]

In [322]:
merged_exposure_df = merged_exposure_df.drop(columns=['type'])

In [323]:
df_combined = (df_combined.merge(merged_exposure_df, left_on=['ISO3', 'adm0_name'], right_on=['iso3', 'adm0_name'], how='left'))

In [324]:
df_combined = df_combined.rename(columns={'population_total': 'total_pop'})

In [325]:
# --- Adding P2 data
vul_path = "{}/p2_vulnerability".format(data_dir)
total_population_file = "{}/p1_exposure/agricultural_drought_fao_1984-2023_exposure_adm0.csv".format(data_dir)

# Load total child population data
total_pop_df = pd.read_csv(total_population_file, usecols=['iso3', 'adm0_name', 'child_population_total'])
total_pop_df = total_pop_df.rename(columns={'iso3': 'ISO3'})

# Ensure unique ISO3-name pairs before merging
total_pop_df = total_pop_df.groupby(['ISO3', 'adm0_name'], as_index=False).agg({'child_population_total': 'mean'})

# Get all CSV files for exposure
p2_vul_files = glob.glob(os.path.join(vul_path, "*.csv"))

# Initialize empty list for processed data
vul_data_list = []

for file in p2_vul_files:
    df = pd.read_csv(file)  # Read full file to check available columns
    if 'iso3' not in df.columns or 'value' not in df.columns:
        continue  # Skip files missing required columns
    # Normalize 'value' column
    df = df[['iso3', 'value']].dropna()
    df = df.rename(columns={'iso3': 'ISO3'}, errors='ignore')
    filename_only = os.path.basename(file)
    hazard_name = '_'.join(filename_only.split('.csv')[:1])  # Extract hazard name
    if hazard_name == 'P2_Child_Mortality':
        continue

    # merge with population data
    df = (df.merge(total_pop_df, on='ISO3', how='left'))

    # rename to hazard and norm
    df['{}'.format(hazard_name)] = np.where(
        (df['child_population_total'] > 0) & (~df['child_population_total'].isna()),
        (df['value']),
        0
    )

    # Rename for relative
    df = df.drop(columns=['child_population_total', 'time_period', 'data_source', 'value'], errors='ignore')

    vul_data_list.append(df)

In [326]:
for i in range(len(vul_data_list)):
    df_combined = (df_combined.merge(vul_data_list[i], on=['ISO3', 'adm0_name'], how='left'))

In [327]:
# Add fragile
fragile['fragile'] = 'fragile'
df_combined = (df_combined.merge(fragile[['ISO3','fragile']], on=['ISO3'], how='left'))

In [328]:
df_combined = df_combined.drop(columns=['iso3'])

In [329]:
# Add components
# Rename
p1_components = p1_components.rename(columns={'river_flood_gmean': 'P1_rfl', 'coastal_flood_gmean': 'P1_cfl',
                                             'storm_gmean': 'P1_ts', 'drought_gmean': 'P1_dr', 'heat_gmean': 'P1_hw',
                                             'fire_gmean': 'P1_fr', 'sand_dust_gmean': 'P1_sds', 'air_pollution_gmean': 'P1_pm25',
                                             'malaria_gmean': 'P1_mal'})

p2_components = p2_components.rename(columns={'health': 'P2_hea', 'nutrition': 'P2_nut', 'education': 'P2_edu',
                                             'protection': 'P2_pro', 'poverty': 'P2_pov', 'survival': 'P2_sur'})

In [330]:
df_combined = (df_combined.merge(p1_components, left_on=['ISO3', 'adm0_name'], right_on=['iso3', 'adm0_name'], how='left'))

In [331]:
df_combined = (df_combined.merge(p2_components, left_on=['ISO3'], right_on=['iso3'], how='left'))

In [332]:
df_combined = df_combined.drop(columns=['adm0_name'])

In [333]:
# Remove any columns we don't want and rearrange as well
# -- Dropping
# Drop min, max
df_combined = df_combined.drop(df_combined.filter(regex='max').columns, axis=1)
df_combined = df_combined.drop(df_combined.filter(regex='min').columns, axis=1)

# -- Renaming
df_combined = df_combined.rename(columns={'name': 'adm_name', 'P1_P2_geometric_avg': 'ccri', 'ISO3':'iso3'})

In [334]:
# Set to 2 decimal places
for col in df_combined.columns:
    if type(df_combined['{}'.format(col)].iloc[0]) != str:
        if col in ['wb_income', 'unicef_ro', 'geometry', 'fragile']:
            continue
        else:
            df_combined[col] = df_combined[col].round(2)

In [335]:
# Rename normalized to _norm
df_combined.columns = [col.replace('_normalized', '_norm') if 'normalized' in col else col for col in df_combined.columns]

In [336]:
# Renaming the hazards
# P1 hazards
df_combined.columns = [col.replace('river_flood', 'rfl') if 'river_flood' in col else col for col in df_combined.columns]
df_combined.columns = [col.replace('coastal_flood', 'cfl') if 'coastal_flood' in col else col for col in df_combined.columns]
df_combined.columns = [col.replace('tropical_storm', 'ts') if 'tropical_storm' in col else col for col in df_combined.columns]
df_combined.columns = [col.replace('agricultural_drought', 'agdr') if 'agricultural_drought' in col else col for col in df_combined.columns]
df_combined.columns = [col.replace('drought_spei', 'metdr_spei') if 'drought_spei' in col else col for col in df_combined.columns]
df_combined.columns = [col.replace('drought_spi', 'metdr_spi') if 'drought_spi' in col else col for col in df_combined.columns]
df_combined.columns = [col.replace('heatwave_frequency', 'hw_fre') if 'heatwave_frequency' in col else col for col in df_combined.columns]
df_combined.columns = [col.replace('heatwave_severity', 'hw_sev') if 'heatwave_severity' in col else col for col in df_combined.columns]
df_combined.columns = [col.replace('heatwave_duration', 'hw_dur') if 'heatwave_duration' in col else col for col in df_combined.columns]
df_combined.columns = [col.replace('extreme_heat', 'ext') if 'extreme_heat' in col else col for col in df_combined.columns]
df_combined.columns = [col.replace('fire_frequency', 'fr_fre') if 'fire_frequency' in col else col for col in df_combined.columns]
df_combined.columns = [col.replace('fire_FRP', 'fr_int') if 'fire_FRP' in col else col for col in df_combined.columns]
df_combined.columns = [col.replace('sand_dust', 'sds') if 'sand_dust' in col else col for col in df_combined.columns]
df_combined.columns = [col.replace('vectorborne_malariapv', 'mal_pv') if 'vectorborne_malariapv' in col else col for col in df_combined.columns]
df_combined.columns = [col.replace('vectorborne_malariapf', 'mal_pf') if 'vectorborne_malariapf' in col else col for col in df_combined.columns]
df_combined.columns = [col.replace('air_pollution', 'pm25') if 'air_pollution' in col else col for col in df_combined.columns]

# P2 hazards
df_combined.columns = [col.replace('P2_Immunization_DTP1', 'hea_dtp1') if 'P2_Immunization_DTP1' in col else col for col in df_combined.columns]
df_combined.columns = [col.replace('P2_Immunization_DTP3', 'hea_dtp3') if 'P2_Immunization_DTP3' in col else col for col in df_combined.columns]
df_combined.columns = [col.replace('P2_Birth_Attendant_Y15T19', 'hea_skat') if 'P2_Birth_Attendant_Y15T19' in col else col for col in df_combined.columns]
df_combined.columns = [col.replace('P2_electricity_access', 'hea_elec') if 'P2_electricity_access' in col else col for col in df_combined.columns]
df_combined.columns = [col.replace('P2_Nutrition_Stunting_Modeled', 'nut_stu') if 'P2_Nutrition_Stunting_Modeled' in col else col for col in df_combined.columns]
df_combined.columns = [col.replace('P2_food_poverty', 'nut_fpov') if 'P2_food_poverty' in col else col for col in df_combined.columns]
df_combined.columns = [col.replace('P2_WASH_Drinking_Water', 'wash_wat') if 'P2_WASH_Drinking_Water' in col else col for col in df_combined.columns]
df_combined.columns = [col.replace('P2_WASH_Drinking_Sanitation', 'wash_san') if 'P2_WASH_Drinking_Sanitation' in col else col for col in df_combined.columns]
df_combined.columns = [col.replace('P2_WASH_Sanitation', 'wash_san') if 'P2_WASH_Sanitation' in col else col for col in df_combined.columns]
df_combined.columns = [col.replace('P2_basic_hygiene', 'wash_hyg') if 'P2_basic_hygiene' in col else col for col in df_combined.columns]
df_combined.columns = [col.replace('P2_LSCED', 'edu_lsos') if 'P2_LSCED' in col else col for col in df_combined.columns]
df_combined.columns = [col.replace('P2_ED_CR_L2', 'edu_lscr') if 'P2_ED_CR_L2' in col else col for col in df_combined.columns]
df_combined.columns = [col.replace('P2_Learning_Poverty', 'edu_lpov') if 'P2_Learning_Poverty' in col else col for col in df_combined.columns]
df_combined.columns = [col.replace('P2_PT_Labor', 'pro_lab') if 'P2_PT_Labor' in col else col for col in df_combined.columns]
df_combined.columns = [col.replace('P2_Child_Marriage', 'pro_mar') if 'P2_Child_Marriage' in col else col for col in df_combined.columns]
df_combined.columns = [col.replace('P2_Child_poverty', 'prov_md') if 'P2_Child_poverty' in col else col for col in df_combined.columns]
df_combined.columns = [col.replace('P2_Social_Protection', 'prov_u5sp') if 'P2_Social_Protection' in col else col for col in df_combined.columns]
df_combined.columns = [col.replace('P2_Under5_Mortality', 'sur_u5mor') if 'P2_Under5_Mortality' in col else col for col in df_combined.columns]

In [337]:
df_combined.columns = [col.replace('value_norm', 'norm') if 'value_norm' in col else col for col in df_combined.columns]

In [338]:
# Rename some things
df_combined = df_combined.rename(columns={'name': 'adm_name', 'ISO3':'iso3'})


In [339]:
df_combined = df_combined[[
    'iso3',
    'adm_name',
    'total_pop',
    'u18_pop',
    'wb_income',
    'unicef_ro',
    'ucode',
    'uuid',
    'geometry',
    'type',
    'fragile','rfl_abs',
 'rfl_rel','rfl_abs_norm',
 'rfl_rel_norm','cfl_abs',
 'cfl_rel','cfl_abs_norm',
 'cfl_rel_norm','ts_abs',
 'ts_rel','ts_abs_norm',
 'ts_rel_norm','agdr_abs',
 'agdr_rel','agdr_abs_norm',
 'agdr_rel_norm','metdr_spei_abs',
 'metdr_spei_rel','metdr_spei_abs_norm',
 'metdr_spei_rel_norm','metdr_spi_abs',
 'metdr_spi_rel','metdr_spi_abs_norm',
 'metdr_spi_rel_norm','hw_fre_abs',
 'hw_fre_rel','hw_fre_abs_norm',
 'hw_fre_rel_norm','hw_dur_abs',
 'hw_dur_rel','hw_dur_abs_norm',
 'hw_dur_rel_norm','hw_sev_abs',
 'hw_sev_rel','hw_sev_abs_norm',
 'hw_sev_rel_norm','ext_abs',
 'ext_rel','ext_abs_norm',
 'ext_rel_norm','fr_fre_abs',
 'fr_fre_rel','fr_fre_abs_norm',
 'fr_fre_rel_norm','fr_int_abs',
 'fr_int_rel','fr_int_abs_norm',
 'fr_int_rel_norm','sds_abs',
 'sds_rel','sds_abs_norm',
 'sds_rel_norm','pm25_abs',
 'pm25_rel','pm25_abs_norm',
 'pm25_rel_norm','mal_pv_abs',
 'mal_pv_rel','mal_pv_abs_norm',
 'mal_pv_rel_norm','mal_pf_abs',
 'mal_pf_rel','mal_pf_abs_norm',
 'mal_pf_rel_norm','hea_dtp1','hea_dtp1_norm', 'hea_dtp3','hea_dtp3_norm','hea_skat','hea_skat_norm',
'hea_elec','hea_elec_norm','nut_stu','nut_stu_norm','nut_fpov','nut_fpov_norm','wash_wat',
    'wash_wat_norm','wash_san','wash_san_norm','wash_hyg','wash_hyg_norm','edu_lsos','edu_lsos_norm',
    'edu_lscr','edu_lscr_norm','edu_lpov','edu_lpov_norm','pro_lab','pro_lab_norm','pro_mar',
    'pro_mar_norm','prov_md','prov_md_norm','prov_u5sp','prov_u5sp_norm','sur_u5mor',
    'P1_rfl', 'P1_cfl', 'P1_ts', 'P1_dr', 'P1_hw', 'P1_fr', 'P1_sds', 'P1_pm25', 'P1_mal',
    'P2_hea', 'P2_nut', 'P2_edu', 'P2_pro', 'P2_pov', 'P2_sur','P1_geometric_avg','P2_arithmetic_avg',
'ccri']]

In [340]:
df_combined

,iso3,adm_name,total_pop,u18_pop,wb_income,unicef_ro,ucode,uuid,geometry,type,...,P1_mal,P2_hea,P2_nut,P2_edu,P2_pro,P2_pov,P2_sur,P1_geometric_avg,P2_arithmetic_avg,ccri
0,SMR,San Marino,32591.89,5133.70,WB_HI,NaN,SMR_V1,a01626d4-992d-4892-af4d-4cdbfb40f0e1,"POLYGON ((12.41147 43.90129, 12.44922 43.8989,...",State,...,NaN,0.81,NaN,NaN,NaN,NaN,0.00,7.53,0.26,2.80
1,MCO,Monaco,24611.52,5150.67,WB_HI,NaN,MCO_V1,f9260923-3a04-4ec8-95eb-741f3036e821,"POLYGON ((7.4117 43.73428, 7.41831 43.7246, 7....",State,...,NaN,0.00,NaN,NaN,NaN,NaN,0.12,NaN,0.00,NaN
2,VAT,Holy See,572.08,16.46,NaN,NaN,VAT_V1,dab41bf4-9504-44b8-a144-2cb92e71a5f5,"POLYGON ((12.4459 41.90193, 12.45631 41.90037,...",State,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,MUS,Mauritius,1248307.52,241215.58,WB_UMI,NaN,MUS_V1,ba4ea616-9799-45c5-b83b-1c6b459716b5,"MULTIPOLYGON (((63.32207 -19.73689, 63.39515 -...",State,...,NaN,0.20,1.52,NaN,NaN,8.92,1.24,7.94,2.71,6.22
4,AND,Andorra,81429.70,12314.43,WB_HI,NaN,AND_V1,0edb2089-1dfc-42cd-a7d8-ccdd7456b358,"POLYGON ((1.44256 42.60367, 1.44601 42.52099, ...",State,...,NaN,0.00,NaN,NaN,NaN,8.78,0.10,7.83,2.48,6.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
190,BOL,Bolivia (Plurinational State of),12331845.09,4403179.14,WB_LMI,UNICEF_LACRO,BOL_V1,77a7e4d5-6ca3-4e3a-a19c-3978442c8f66,"POLYGON ((-62.6452 -22.25046, -62.62124 -22.17...",State,...,6.17,3.28,1.32,1.28,3.28,3.72,1.98,6.22,3.06,5.97
191,URY,Uruguay,3336010.78,769020.36,WB_HI,UNICEF_LACRO,URY_V1,96f0a21b-d2a0-4072-b394-2a8e445b42d6,"POLYGON ((-58.43656 -33.52902, -58.43117 -33.6...",State,...,NaN,0.19,0.80,4.47,NaN,3.44,0.47,6.62,1.76,5.06
192,PRY,Paraguay,7026327.62,2365287.67,WB_UMI,UNICEF_LACRO,PRY_V1,0d68a9a7-75e6-4094-be23-e64d89ee6d51,"POLYGON ((-62.6452 -22.25046, -62.61168 -22.27...",State,...,NaN,2.46,0.75,3.84,4.00,4.58,1.42,6.82,2.88,6.03
193,ARG,Argentina,44999932.97,11966532.09,WB_UMI,UNICEF_LACRO,ARG_V1,7fed002b-6a83-4ec8-9828-83e027917b1b,"MULTIPOLYGON (((-62.07879 -39.07597, -62.07531...",State,...,NaN,2.79,1.70,2.88,1.42,1.22,0.76,4.42,1.77,4.43


In [341]:
# int vals that are abs
for col in df_combined.columns:
    if 'abs' in col and 'norm' not in col:
        df_combined[col] = df_combined[col].astype(int)

In [342]:
df_combined['total_pop'] = df_combined['total_pop'].round(0)
df_combined['u18_pop'] = df_combined['u18_pop'].round(0)
df_combined['total_pop'] = pd.to_numeric(df_combined['total_pop'], errors='coerce').astype('Int64')
df_combined['u18_pop'] = pd.to_numeric(df_combined['u18_pop'], errors='coerce').astype('Int64')

In [343]:
gdf = gpd.GeoDataFrame(df_combined, geometry=df_combined['geometry'], crs='EPSG:4326')

In [344]:
gdf.to_file('{}/CCRI_results_misc/CCRI_P1_P2_format.geojson'.format(data_dir))